Set up env

In [ ]:
import carla
import numpy as np
from src.simulator import Simulator
from src.agent import NCPAgent
from src.model import Model, Trainer

import torchvision
from IPython.display import display
import sys
import os
import torch

sys.path.append("CARLA_SIM/PythonAPI/carla/")
from agents.navigation.basic_agent import BasicAgent


# Training process

In [ ]:
from src.model import Model, Trainer

In [ ]:
simulator = Simulator(world_name='Town04_opt', dump_data=False)

In [ ]:
import time

simulator.world.get_spectator()#.set_transform()
    # carla.Transform(
    #     location=carla.Location(x=398.7934265136719,
    #                             y=-56.03200912475586,
    #                             z=3.37939715385437)))

simulator.spawn_car_with_camera(
    rel_coordinates=carla.Location(x=1.2, z=1.9) # camera coords
)
vehicle = simulator.get_vehicle()


output_size = 2
units = 19
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

ncp = Model(output_size, units)
ncp.to(device)
if not os.path.isdir(f'out/{simulator.world_name}'):
    os.mkdir(f'out/{simulator.world_name}')
with open(f'out/{simulator.world_name}/data.txt', 'a+') as f:
    f.write(f'timestamp start = {time.time()}\n')
agent = NCPAgent(simulator, ncp, target_speed=10)

next_waypoint  = [simulator.world.get_map().get_waypoint(vehicle.get_location(),
                                                    project_to_road=True,
                                                    lane_type=(carla.LaneType.Driving))]

waypoints = []
dist_between_waypoints = 15
# waypoint_num = 350
waypoint_num = 100
# waypoint_num = 10
for _ in range(waypoint_num):
    waypoints.append(next_waypoint[-1])
    # simulator.world.get_spectator().set_transform(next_waypoint[-1].transform)
    next_waypoint = next_waypoint[-1].next(dist_between_waypoints)

dest_idx = 2
dest = waypoints[dest_idx].transform.location
agent.set_destination(dest)
agent.set_target_speed(10)

In [ ]:
# # os.rmdir('./out/')

idx = 1
tmp = 0
waypoint = waypoints[idx]
to_PIL = torchvision.transforms.ToPILImage()

# loss_func = torch.nn.functional.mse_loss
# optimizer = torch.optim.Adam(ncp.parameters(), lr=0.001)

# # basic_agent = BasicAgent(vehicle=vehicle)
# trainer = Trainer(ncp, loss_func, optimizer)

while True:
    tmp += 1
    if vehicle.get_location().distance(dest) < 0.5:
        print(f"Destination reached")
        break
    if (vehicle.get_location().distance(waypoint.transform.location) <= dist_between_waypoints / 4) or \
        (vehicle.get_location().distance(waypoint.transform.location) >= dist_between_waypoints * 2 and \
         vehicle.get_location().distance(waypoint.transform.location) <= dist_between_waypoints * 3):
        print(f'Waypoint {idx} was reached')
        waypoint = waypoints[idx + 1]
        idx += 1

    # control, movement, raw_data, out_tensor = agent.run_step()
    control, _, raw_data= agent.run_step()
    # if raw_data is not None:
    #     trainer.train(raw_data, torch.tensor(control.steer))
    #     print(out_tensor)
    if tmp % 150000:
        print(f"control = {control}")
    vehicle.apply_control(control)
    if agent.simulator.image_frame is not None:
        with open(f'out/{simulator.world_name}/data.txt', 'a+') as f:
            f.write(f'{agent.simulator.image_frame} : {control.steer}\n')

    
    if agent.done():
        if dest_idx < waypoint_num - 1:
            dest_idx += 10
            dest_idx = min(dest_idx, waypoint_num - 1)
            print(f'Intermediate destination reached. Moving to waypoint {dest_idx}')
            agent.is_done = False
            agent.set_destination(waypoints[dest_idx].transform.location)
            continue

        print("The target has been reached, stopping the simulation")
        break
vehicle.apply_control(carla.VehicleControl(throttle = 0.0, brake=1.0, steer = 0.0))

In [ ]:
simulator.destroy_all()

# training

In [1]:
from src.model import Model, Trainer

import torch

output_size = 4
units = 19
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

model = Model(output_size, units)
model.to(device)

loss_func = torch.nn.functional.mse_loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

data_path = "out/Town04_opt"
trainer = Trainer(model,
                  loss_func,
                  optimizer,
                  annotations_file=f"{data_path}/data.txt",
                  img_dir=data_path,
                  test_size=0.3,
                  random_state=42,
                  stb_weights=[1, 0, 0])
trainer.train(epochs=25, batch_size=16)


cuda
alloc!
Epoch 0


Train:   0%|                                                                                                                                                                                              | 0/224 [00:00<?, ?it/s]/mnt/hard_drive/habkaffee_part/programming/masters_paper/venv/lib/python3.8/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Test: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:04<00:00, 21.85it/s]


LOSS train 0.005941047333180904 valid 0.001487386180087924
Epoch 1


Test: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:04<00:00, 21.47it/s]


LOSS train 0.004689883440732956 valid 0.0013573496835306287
Epoch 2


...

LOSS train 0.0004366279754322022 valid 0.000301057705655694
Epoch 23


Test: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:04<00:00, 22.04it/s]


LOSS train 0.0034448003862053156 valid 0.00032519601518288255
Epoch 24


Test: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:04<00:00, 22.14it/s]

LOSS train 0.00010517596092540771 valid 0.00020578844123519957


In [3]:
model.save_model(f'model/Town04_opt_model_test25.pth')

In [ ]:
# ncp.save_model(f'./model/pretrained_wp{waypoint_num}.pth')

# test

In [4]:
import carla
import numpy as np
from src.simulator import Simulator
from src.agent import NCPAgent
from src.model import Model, Trainer

from random import uniform

import torchvision
from IPython.display import display
import sys
import os
import torch
sys.path.append("CARLA_SIM/PythonAPI/carla/")
from agents.navigation.basic_agent import BasicAgent


In [5]:
simulator = Simulator(world_name='Town04_opt', debug=False)

In [6]:
simulator.spawn_car_with_camera(
    rel_coordinates=carla.Location(x=1.2, z=1.9), # camera coords
    vehicle_rotation=(0, uniform(-91, -89), 0)
)
vehicle = simulator.get_vehicle()
print(f'Transforms : {vehicle.get_transform()}')

output_size = 4
units = 19
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

Transforms : Transform(Location(x=405.320374, y=-48.450779, z=0.218632), Rotation(pitch=0.000000, yaw=-90.833366, roll=0.000000))
cuda


In [7]:
ncp = Model(output_size, units)
ncp.load_model('model/Town04_opt_model_test25.pth')
ncp = ncp.to(device=device)
ncp.eval()
agent = NCPAgent(simulator, ncp, target_speed=10)

alloc!
Camera stream started


In [8]:
# ncp = Model(output_size, units)
# ncp = ncp.to(device=device)
# agent = NCPAgent(simulator, ncp, target_speed=10)

next_waypoint  = [simulator.world.get_map().get_waypoint(vehicle.get_location(),
                                                    project_to_road=True,
                                                    lane_type=(carla.LaneType.Driving))]

waypoints = []
dist_between_waypoints = 20
waypoint_num = 5
for _ in range(waypoint_num):
    waypoints.append(next_waypoint[-1])
    next_waypoint = next_waypoint[-1].next(dist_between_waypoints)

dest_idx = 2
dest = waypoints[dest_idx].transform.location
agent.set_destination(dest)
agent.ignore_traffic_lights(True)
agent.ignore_stop_signs(True)
agent.set_target_speed(10)

idx = 0
while True:
    control, out, raw_data = agent.run_step()
    print(control.steer, out[0][:2], out[0][1]-out[0][0])
    # break
    new_control = carla.VehicleControl(steer=control.steer, throttle=control.throttle, brake=control.brake)
    vehicle.apply_control(new_control)

    if agent.simulator.image_frame is not None:
        with open(f'out/{simulator.world_name}/data.txt', 'a+') as f:
            f.write(f'{agent.simulator.image_frame} : {control.steer}\n')

    if agent.done():
        if dest_idx < waypoint_num - 1:
            dest_idx += 10
            dest_idx = min(dest_idx, waypoint_num - 1)
            print(f'Intermediate destination reached. Moving to waypoint {dest_idx}')
            agent.is_done = False
            agent.set_destination(waypoints[dest_idx].transform.location)
            continue

        print("The target has been reached, stopping the simulation")
        break
    idx += 1
vehicle.apply_control(carla.VehicleControl(throttle = 0.0, brake=1.0, steer = 0.0))
simulator.destroy_all()

0.043529242277145386 tensor([0.0128, 0.0203], device='cuda:0', grad_fn=<SliceBackward0>) tensor(0.0075, device='cuda:0', grad_fn=<SubBackward0>)
0.0436408556997776 tensor([0.0074, 0.0164], device='cuda:0', grad_fn=<SliceBackward0>) tensor(0.0090, device='cuda:0', grad_fn=<SubBackward0>)
0.04369666054844856 tensor([0.0069, 0.0164], device='cuda:0', grad_fn=<SliceBackward0>) tensor(0.0095, device='cuda:0', grad_fn=<SubBackward0>)
0.04375246912240982 tensor([0.0069, 0.0164], device='cuda:0', grad_fn=<SliceBackward0>) tensor(0.0095, device='cuda:0', grad_fn=<SubBackward0>)
0.04380827397108078 tensor([0.0070, 0.0161], device='cuda:0', grad_fn=<SliceBackward0>) tensor(0.0092, device='cuda:0', grad_fn=<SubBackward0>)
0.04386408254504204 tensor([0.0068, 0.0160], device='cuda:0', grad_fn=<SliceBackward0>) tensor(0.0092, device='cuda:0', grad_fn=<SubBackward0>)
0.043919887393713 tensor([0.0068, 0.0160], device='cuda:0', grad_fn=<SliceBackward0>) tensor(0.0092, device='cuda:0', grad_fn=<SubBackwa

In [6]:
simulator.destroy_all()

Camera stream stopped
